In [1]:
import numpy as np
import pandas as pd

In [2]:
root='/Genomics/pritykinlab/zzhao/metabolic_analysis'
reaction_meta_file = f'{root}/Compass/compass/Resources/Recon2_export/rxn_md.csv'
reaction_metadata = pd.read_csv(reaction_meta_file, index_col = 0)
reaction_metadata

,rxn_name_long,subsystem,ind_in_original_recon,rxn_EC_number,genes_associated_with_rxn,rxn_formula,rxn_confidence
rxn_code_nodirection,,,,,,,
10FTHF5GLUtl,"5-glutamyl-10FTHF transport, lysosomal","Transport, lysosomal",1,NaN,NaN,1.00 * 10-formyltetrahydrofolate-[Glu](5) [c] ...,2.0
10FTHF5GLUtm,"5-glutamyl-10FTHF transport, mitochondrial","Transport, mitochondrial",2,NaN,NaN,1.00 * 10-formyltetrahydrofolate-[Glu](5) [m] ...,2.0
10FTHF6GLUtl,"6-glutamyl-10FTHF transport, lysosomal","Transport, lysosomal",3,NaN,NaN,1.00 * 10-formyltetrahydrofolate-[Glu](6) [c] ...,2.0
10FTHF6GLUtm,"6-glutamyl-10FTHF transport, mitochondrial","Transport, mitochondrial",4,NaN,NaN,1.00 * 10-formyltetrahydrofolate-[Glu](6) [m] ...,2.0
10FTHF7GLUtl,"7-glutamyl-10FTHF transport, lysosomal","Transport, lysosomal",5,NaN,NaN,1.00 * 10-formyltetrahydrofolate-[Glu](7) [c] ...,2.0
...,...,...,...,...,...,...,...
MAL_Lte,Transport of L-malate,"Transport, extracellular",7436,NaN,NaN,1.00 * (S)-malate(2-) [e] --> 1.00 * (S)-malat...,NaN
OROTGLUt,OROTGLUt,"Transport, extracellular",7437,NaN,SLC22A7,1.00 * L-glutamate(1-) [c] + 1.00 * Orotate [e...,NaN
PNTOte,Transport of (R)-Pantothenate,"Transport, extracellular",7438,NaN,NaN,1.00 * (R)-Pantothenate [e] --> 1.00 * (R)-Pan...,NaN


In [3]:
# confidence score (4 = most confident; 0 = unassigned confidence) 
reaction_metadata = reaction_metadata[reaction_metadata['rxn_confidence'].isin([1,2, 3, 4])]
# Enzyme Commission Number (EC Number)
reaction_metadata = reaction_metadata[~reaction_metadata['rxn_EC_number'].isna()]
# gene names
reaction_metadata = reaction_metadata[~reaction_metadata['genes_associated_with_rxn'].isna()]
# filter out reactions in the citric acid cycle subsystem which are outside of the mitochondria. remove if '[m]' not in the rxn_formula 
reaction_metadata.loc[(reaction_metadata['rxn_formula'].map(lambda x: '[m]' not in x)) & (reaction_metadata['subsystem'] == "Citric acid cycle"), 'subsystem'] = 'Other'
reaction_metadata = reaction_metadata[reaction_metadata['subsystem']!='Other']


In [10]:
reaction_metadata.loc[reaction_metadata.index[0]]

rxn_name_long                1,3-Diaminopropane:oxygen oxidoreductase (deam...
subsystem                                              beta-Alanine metabolism
ind_in_original_recon                                                       13
rxn_EC_number                                                          1.4.3.6
genes_associated_with_rxn                                     AOC1; AOC2; AOC3
rxn_formula                  1.00 * trimethylenediaminium [c] + 1.00 * Wate...
rxn_confidence                                                             1.0
Name: 13DAMPPOX, dtype: object

In [16]:
reaction_to_genes = {}
for key in reaction_metadata.index:
    gene_list = reaction_metadata.loc[key]['genes_associated_with_rxn'].split('; ')
    reaction_to_genes[key] = (gene_list, reaction_metadata.loc[key]['subsystem'])

In [29]:
import joblib
joblib.dump(reaction_to_genes, 'gene_signatures/metabolism_rxn_reaction.pickle')

['gene_signatures/metabolism_rxn_reaction.pickle']

In [23]:
from collections import Counter
c = 0
for item in Counter([len(reaction_to_genes[key][0]) for key in reaction_to_genes]).most_common():
    if item[0] >= 5:
        c += item[1]
c

100

In [27]:
reaction_metadata[reaction_metadata.apply(lambda x: len(x['genes_associated_with_rxn'].split('; ')) >= 5, axis=1)]['subsystem'].value_counts()

Keratan sulfate synthesis                    18
Nucleotide interconversion                   13
Inositol phosphate metabolism                 9
Pyruvate metabolism                           8
Vitamin A metabolism                          4
Glyoxylate and dicarboxylate metabolism       4
Glycerophospholipid metabolism                4
Chondroitin synthesis                         4
Pyrimidine catabolism                         3
Lysine metabolism                             3
Histidine metabolism                          2
Fructose and mannose metabolism               2
Glutathione metabolism                        2
Tyrosine metabolism                           2
Propanoate metabolism                         2
Fatty acid oxidation                          2
beta-Alanine metabolism                       2
Oxidative phosphorylation                     2
Triacylglycerol synthesis                     2
Aminosugar metabolism                         1
Transport, lysosomal                    

In [18]:
Counter([reaction_to_genes[key][1] for key in reaction_to_genes]).most_common()


[('Fatty acid oxidation', 149),
 ('Nucleotide interconversion', 130),
 ('Keratan sulfate degradation', 59),
 ('Tyrosine metabolism', 46),
 ('Sphingolipid metabolism', 46),
 ('Keratan sulfate synthesis', 41),
 ('Steroid metabolism', 39),
 ('Chondroitin synthesis', 36),
 ('Tryptophan metabolism', 33),
 ('Glycerophospholipid metabolism', 32),
 ('Inositol phosphate metabolism', 32),
 ('Folate metabolism', 30),
 ('Valine, leucine, and isoleucine metabolism', 28),
 ('Cholesterol metabolism', 26),
 ('Chondroitin sulfate degradation', 26),
 ('Arginine and Proline Metabolism', 25),
 ('Bile acid synthesis', 25),
 ('Fatty acid synthesis', 23),
 ('Glycine, serine, alanine and threonine metabolism', 22),
 ('Pyrimidine catabolism', 21),
 ('Blood group synthesis', 20),
 ('Aminosugar metabolism', 19),
 ('Pyruvate metabolism', 19),
 ('Heparan sulfate degradation', 18),
 ('Purine catabolism', 17),
 ('Urea cycle', 17),
 ('Eicosanoid metabolism', 17),
 ('Methionine and cysteine metabolism', 15),
 ('Lysine

In [68]:
from collections import Counter
subsystem_to_genes = {}
for key in reaction_metadata['subsystem'].value_counts().keys():
    reaction_metadata[reaction_metadata['subsystem']==key]['genes_associated_with_rxn']
    gene_list = []
    for item in reaction_metadata[reaction_metadata['subsystem']==key]['genes_associated_with_rxn'].values:
        gene_list.extend(item.split('; '))
    subsystem_to_genes[key] = Counter(gene_list).most_common()

In [72]:
subsystem_to_genes.keys()

dict_keys(['Fatty acid oxidation', 'Nucleotide interconversion', 'Keratan sulfate degradation', 'Sphingolipid metabolism', 'Tyrosine metabolism', 'Keratan sulfate synthesis', 'Steroid metabolism', 'Chondroitin synthesis', 'Tryptophan metabolism', 'Glycerophospholipid metabolism', 'Inositol phosphate metabolism', 'Folate metabolism', 'Valine, leucine, and isoleucine metabolism', 'Chondroitin sulfate degradation', 'Cholesterol metabolism', 'Arginine and Proline Metabolism', 'Bile acid synthesis', 'Fatty acid synthesis', 'Glycine, serine, alanine and threonine metabolism', 'Pyrimidine catabolism', 'Blood group synthesis', 'Pyruvate metabolism', 'Aminosugar metabolism', 'Heparan sulfate degradation', 'Urea cycle', 'Purine catabolism', 'Eicosanoid metabolism', 'Methionine and cysteine metabolism', 'Lysine metabolism', 'Starch and sucrose metabolism', 'Pentose phosphate pathway', 'NAD metabolism', 'Vitamin A metabolism', 'beta-Alanine metabolism', 'Triacylglycerol synthesis', 'Glyoxylate and

In [70]:
subsystem_to_genes['Fatty acid oxidation']

[('CPT1A', 34),
 ('CPT1B', 33),
 ('CPT1C', 33),
 ('ACSL1', 28),
 ('ACADM', 28),
 ('ACADS', 26),
 ('ACAA1B', 23),
 ('ACOX1', 18),
 ('EHHADH', 17),
 ('HSD17B4', 17),
 ('ACSL3', 10),
 ('OLAH', 8),
 ('CRAT', 7),
 ('ACSL4', 7),
 ('ACSBG2', 5),
 ('CPT2', 4),
 ('FASN', 3),
 ('ACSL5', 3),
 ('SLC27A2', 2),
 ('LIPC', 2),
 ('ACACB', 1),
 ('CROT', 1),
 ('ECI1', 1),
 ('ECI3', 1),
 ('ETFDH', 1),
 ('ACSL6', 1),
 ('ACSM1', 1),
 ('ALDH1L1', 1),
 ('LCAT', 1),
 ('MLYCD', 1),
 ('MCAT', 1),
 ('PHYH', 1)]

In [73]:
term_to_genes = {key: [item[0] for item in subsystem_to_genes[key]] for key in subsystem_to_genes}

In [75]:
import joblib
joblib.dump(subsystem_to_genes, 'gene_signatures/metabolism_rxn_subsystem.pickle')

['gene_signatures/metabolism_rxn_subsystem.pickle']